tf.data支持常用数据的读取，常用的api里有batch,zip,shuffle,prefetch,map,iterleave等,它们的排列组合使用情况很多，官方文档没有详细的说，看过tensorflow官方人员的讲解，但是只说了不同的顺序有不同的效果，具体什么效果他们也不好下结论。本文将做个小实验，来初步探索一下这个过程，目的是以后能正确使用，最基本的是来保证读入数据正确，如果能正确的基础上高效使用那是更好的。

# 1、顺序对结果的影响

 产生实验用数据,并将数扰分加存到四个文件中，这样可以实现用tf.data来完成从多个文件中读取的过程

In [81]:
import numpy as np
import os
from collections import Counter
from absl import logging
import time
import tensorflow as tf
print(tf.__version__)

2.1.0


In [18]:
datas = np.arange(400,2000).reshape(400,4) # 400个样本，每个样本有4个特征
labels = np.arange(400).reshape(400,1) #每个样本的序号，方便分析数据变化
ld = np.concatenate((labels,datas),axis=1).astype(np.float32)#把数据转换成float32
print(ld[:10])

[[  0. 400. 401. 402. 403.]
 [  1. 404. 405. 406. 407.]
 [  2. 408. 409. 410. 411.]
 [  3. 412. 413. 414. 415.]
 [  4. 416. 417. 418. 419.]
 [  5. 420. 421. 422. 423.]
 [  6. 424. 425. 426. 427.]
 [  7. 428. 429. 430. 431.]
 [  8. 432. 433. 434. 435.]
 [  9. 436. 437. 438. 439.]]


接着把数据保存成二进制文件，来适配tf.data实现从文件读数据的流程（而非内存）。

In [19]:
savepath="/tmp/ourtest"
if not os.path.exists(savepath):
    os.makedirs(savepath)#生成路保存数据的路径
with open("/tmp/ourtest/test_0.dat","wb") as f:
    f.write(ld[:100,:].tobytes())
with open("/tmp/ourtest/test_1.dat","wb") as f:
    f.write(ld[100:200,:].tobytes())
with open("/tmp/ourtest/test_2.dat","wb") as f:
    f.write(ld[200:300,:].tobytes())
with open("/tmp/ourtest/test_3.dat","wb") as f:
    f.write(ld[300:,:].tobytes())

In [20]:
!tree /tmp/ourtest

/tmp/ourtest
├── test_0.dat
├── test_1.dat
├── test_2.dat
└── test_3.dat

0 directories, 4 files


## 1.1 实验一：理解map,batch,shuffle,repeat,prefetch的作用

In [51]:
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
print("====================================after fixedlength read==========================================")
print("dataset type",dataset.element_spec)
print("one value of dataset",iter(dataset).next())#dataset是一个生成器，另外读数据方法是 for data in dataset: print data
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
print("====================================after parser==========================================")
print("dataset type",dataset.element_spec)
print("one value of dataset",iter(dataset).next()[0].numpy().flatten())
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=False)
print("====================================after batch read==========================================")
print("dataset type",dataset.element_spec)
print("one value of dataset",iter(dataset).next()[0].numpy().flatten())
dataset = dataset.shuffle(buffer_size=1000,reshuffle_each_iteration=False)
print("====================================after shuffle read==========================================")
print("dataset type",dataset.element_spec)
print("one value of dataset",iter(dataset).next()[0].numpy().flatten())
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
print("====================================after prefetch read==========================================")
print("dataset type",dataset.element_spec)
print("one value of dataset",iter(dataset).next()[0].numpy().flatten())

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
====================================after fixedlength read==========================================
dataset type TensorSpec(shape=(), dtype=tf.string, name=None)
one value of dataset tf.Tensor(b'\x00\x00\x00\x00\x00\x00\xc8C\x00\x80\xc8C\x00\x00\xc9C\x00\x80\xc9C', shape=(), dtype=string)
====================================after parser==========================================
dataset type (TensorSpec(shape=(None,), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))
one value of dataset [0.]
====================================after batch read==========================================
dataset type (TensorSpec(shape=(None, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.float32, name=None))
one value of dataset [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22.

## 实验二 1.2加或不加shuffle和prefetch以及调整顺序等地比实验

### 1.2.1 batch整除总数据量，无shuffle,prefetch

In [68]:
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=False)
#dataset = dataset.shuffle(buffer_size=1000,reshuffle_each_iteration=False)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]
[50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67.
 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85.
 86. 87. 88. 89. 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149.]
[150. 151. 152. 153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163.
 164. 165. 166. 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177.
 178. 179. 180. 181. 182. 183. 184. 185. 186. 187. 188. 189. 190. 191.
 19

### 1.2.2 batch整除总数据量，无shuffle,有prefetch

In [84]:
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=False)
#dataset = dataset.shuffle(buffer_size=1000,reshuffle_each_iteration=False)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]

for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
            
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))


data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]
[50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67.
 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85.
 86. 87. 88. 89. 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149.]
[150. 151. 152. 153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163.
 164. 165. 166. 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177.
 178. 179. 180. 181. 182. 183. 184. 185. 186. 187. 188. 189. 190. 191.
 19

### 1.2.3 batch 不能整除总数据量，，drop_remainder=False，无shuffle,有prefetch

In [71]:
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=51
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=False)
#dataset = dataset.shuffle(buffer_size=1000,reshuffle_each_iteration=False)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[ 51.  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.
  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.
  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.
  93.  94.  95.  96.  97.  98.  99. 100. 101.]
[102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113. 114. 115.
 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127. 128. 129.
 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141. 142. 143.
 144. 145. 146. 147. 148. 149. 150. 151. 152.]
[153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166.
 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180.
 181. 182

### 1.2.4 batch 不能整除总数据量，，drop_remainder=True，无shuffle,有prefetch

In [72]:
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=51
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=True)
#dataset = dataset.shuffle(buffer_size=1000,reshuffle_each_iteration=False)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
[ 51.  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.
  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.
  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.  90.  91.  92.
  93.  94.  95.  96.  97.  98.  99. 100. 101.]
[102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113. 114. 115.
 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127. 128. 129.
 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141. 142. 143.
 144. 145. 146. 147. 148. 149. 150. 151. 152.]
[153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166.
 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180.
 181. 182

### 1.2.5 batch 能整除总数据量，，drop_remainder=False，有shuffle=1、batch、2*batch，有prefetch

In [73]:
#shuffle buffer_size=1
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=True)
dataset = dataset.shuffle(buffer_size=1,reshuffle_each_iteration=False)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]
[50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67.
 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85.
 86. 87. 88. 89. 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149.]
[150. 151. 152. 153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163.
 164. 165. 166. 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177.
 178. 179. 180. 181. 182. 183. 184. 185. 186. 187. 188. 189. 190. 191.
 19

In [75]:
#shuffle buffer_size=batchsize
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=True)
dataset = dataset.shuffle(buffer_size=batchsize,reshuffle_each_iteration=False)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[200. 201. 202. 203. 204. 205. 206. 207. 208. 209. 210. 211. 212. 213.
 214. 215. 216. 217. 218. 219. 220. 221. 222. 223. 224. 225. 226. 227.
 228. 229. 230. 231. 232. 233. 234. 235. 236. 237. 238. 239. 240. 241.
 242. 243. 244. 245. 246. 247. 248. 249.]
[250. 251. 252. 253. 254. 255. 256. 257. 258. 259. 260. 261. 262. 263.
 264. 265. 266. 267. 268. 269. 270. 271. 272. 273. 274. 275. 276. 277.
 278. 279. 280. 281. 282. 283. 284. 285. 286. 287. 288. 289. 290. 291.
 292. 293. 294. 295. 296. 297. 298. 299.]
[350. 351. 352. 353. 354. 355. 356. 357. 358. 359. 360. 361. 362. 363.
 364. 365. 366. 367. 368. 369. 370. 371. 372. 373. 374. 375. 376. 377.
 378. 379. 380. 381. 382. 383. 384. 385. 386. 387. 388. 389. 390. 391.
 392. 393. 394. 395. 396. 397. 398. 399.]
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 12

In [76]:
#shuffle buffer_size=2*batchsize
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset=dataset.batch(batch_size=batchsize,drop_remainder=True)
dataset = dataset.shuffle(buffer_size=2*batchsize,reshuffle_each_iteration=False)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[300. 301. 302. 303. 304. 305. 306. 307. 308. 309. 310. 311. 312. 313.
 314. 315. 316. 317. 318. 319. 320. 321. 322. 323. 324. 325. 326. 327.
 328. 329. 330. 331. 332. 333. 334. 335. 336. 337. 338. 339. 340. 341.
 342. 343. 344. 345. 346. 347. 348. 349.]
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149.]
[350. 351. 352. 353. 354. 355. 356. 357. 358. 359. 360. 361. 362. 363.
 364. 365. 366. 367. 368. 369. 370. 371. 372. 373. 374. 375. 376. 377.
 378. 379. 380. 381. 382. 383. 384. 385. 386. 387. 388. 389. 390. 391.
 392. 393. 394. 395. 396. 397. 398. 399.]
[250. 251. 252. 253. 254. 255. 256. 257. 258. 259. 260. 261. 262. 263.
 264. 265. 266. 267. 268. 269. 270. 271. 27

将shuffle放到batch前边，再次重复上边三种buffer_size的实验

In [78]:
#shuffle buffer_size=1
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset = dataset.shuffle(buffer_size=1,reshuffle_each_iteration=False)
dataset=dataset.batch(batch_size=batchsize,drop_remainder=True)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49.]
[50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67.
 68. 69. 70. 71. 72. 73. 74. 75. 76. 77. 78. 79. 80. 81. 82. 83. 84. 85.
 86. 87. 88. 89. 90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]
[100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111. 112. 113.
 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125. 126. 127.
 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139. 140. 141.
 142. 143. 144. 145. 146. 147. 148. 149.]
[150. 151. 152. 153. 154. 155. 156. 157. 158. 159. 160. 161. 162. 163.
 164. 165. 166. 167. 168. 169. 170. 171. 172. 173. 174. 175. 176. 177.
 178. 179. 180. 181. 182. 183. 184. 185. 186. 187. 188. 189. 190. 191.
 19

In [80]:
#shuffle buffer_size=batchsize
filenames=sorted(tf.io.gfile.glob('/tmp/ourtest/test*')) #获取文件
print("data files:",filenames)
dataset= tf.data.FixedLengthRecordDataset(filenames=filenames,record_bytes=5*np.dtype(np.float32).itemsize)#单个样本的总节节数
def parser(value):
    """
    需要对dataset中的数据进行解码
    """
    record = tf.io.decode_raw(value,out_type=tf.float32)
    label = record[:1]
    data = record[1:]
    return label,data
#map可以叠加多个重复使用,第二个参数是用来加速的，可以使用tf.data.experimental.AUTOTUNE自适应的来做
dataset=dataset.map(parser,num_parallel_calls=None)
batchsize=50
# 第二个参数表示数据不能被batch_size整除，剩下的数据小于batch_size后是否丢弃
dataset = dataset.shuffle(buffer_size=batchsize,reshuffle_each_iteration=False)
dataset=dataset.batch(batch_size=batchsize,drop_remainder=True)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)#这个参数可以指定也可以
iterations=10
total_time=[]
values=[]
for t in range(iterations):
    start = time.time()
    for i,(label,data) in enumerate(dataset):
        if t==0:
            print(label.numpy().flatten())
            values.extend(label.numpy().flatten().tolist())
    duration=time.time()-start
    total_time.append(duration)
print("the average time is %s sec :",str(np.mean(total_time)))
print("statistic of values",Counter(values))

data files: ['/tmp/ourtest/test_0.dat', '/tmp/ourtest/test_1.dat', '/tmp/ourtest/test_2.dat', '/tmp/ourtest/test_3.dat']
[ 5. 22. 42. 38. 28. 16. 15. 48. 30. 23. 49. 12. 29. 19. 24.  7.  3.  6.
 53.  2. 11. 55. 62. 31. 17. 27. 74. 73. 57. 56. 71. 66. 78. 67. 32. 63.
 83. 43. 59. 58.  8. 89. 18. 10. 40. 50.  9. 81. 25. 41.]
[ 26.  98.  45. 101.   1.  65.  79. 104.  93.  61. 108.  77. 105.  14.
  36.  95.  85. 103.  75.  35.  91.  68.  37.  70.  90. 107.  46. 121.
   4.  13. 112. 100.  82.  84. 102. 114. 133.  80.  52.  94.  99.  97.
 141.   0. 120. 144. 106. 130. 123.  39.]
[140. 111. 143.  51.  64. 148.  87. 149. 113. 134.  72.  76.  33. 128.
  21. 153.  86. 151.  96.  69. 116. 150. 157. 145.  88.  60. 168. 166.
 154. 131. 137. 163. 160. 158. 117.  92. 167. 186. 174.  34. 127. 138.
 177. 164. 181. 115. 136.  47. 183. 195.]
[124. 139. 146. 176. 165. 171. 197. 189. 175. 178. 199. 205. 125. 135.
 173. 204.  44. 209. 187. 215. 190. 132. 155. 184. 218. 201. 110. 118.
 179. 213. 222.  20. 22

### 本部分实验结论
- 1.2.1和1.2.2做对比，加prefetch后时间并没有显著提升，可能是数据太小。
- 1.2.3和1.2.4做对比，可以看出drop_remainder可以把不足一个batch大小的数据选择舍弃，显然测试时这个参数一定是False,只能处理固定batch的数据的时候，这个参数要设置成True.
- 

### 1.2.2 batchsize 不整除总数据量（这种情况更加符合实际）

In [56]:
import pandas as pd
from sklearn.metrics import confusion_matrix

In [55]:
import sklearn

In [69]:
pd.crosstab(labels,preds)

col_0,0,1
row_0,,
0,0,1
1,1,1


In [61]:
confusion_matrix(labels,preds)

array([[0, 1],
       [1, 1]])

In [ ]:
tianlianglllll1@foxmail.com